In [1]:
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (Conv2D, SeparableConv2D, BatchNormalization, Activation,
                                     Dropout, AveragePooling2D, GlobalAveragePooling2D,
                                     Input, MaxPooling2D, Flatten)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import cv2
import imutils
import os

In [17]:
dataset_path = 'fer2013/fer2013/fer2013.csv'
image_size = (48, 48)


def load_fer2013():
    data = pd.read_csv(dataset_path)
    pixels = data['pixels'].tolist()
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(48, 48)
        face = cv2.resize(face.astype('uint8'), image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).values
    return faces, emotions


def preprocess_input(x, v2=True):
    x = x.astype('float32') / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

In [12]:
def simple_CNN(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(16, (7, 7), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(16, (7, 7), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Conv2D(32, (5, 5), padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (5, 5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(num_classes, (3, 3), padding='same'))
    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax', name='predictions'))
    return model


In [13]:
def mini_XCEPTION(input_shape, num_classes, l2_regularization=0.01):
    regularization = l2(l2_regularization)

    img_input = Input(input_shape)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    for filters in [16, 32, 64, 128]:
        residual = Conv2D(filters, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
        residual = BatchNormalization()(residual)

        x = SeparableConv2D(filters, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(filters, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
        x = BatchNormalization()(x)

        x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
        x = layers.add([x, residual])

    x = Conv2D(num_classes, (3, 3), padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax', name='predictions')(x)

    return Model(img_input, output)


In [14]:
input_shape = (64, 64, 1)
num_classes = 7
model = simple_CNN((48, 48, 1), num_classes)
model.summary()


C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 16)     │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 16)     │        12,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48, 48, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 48, 48, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 24, 24, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 32)     │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 24, 24, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 32)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 24, 24, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 12, 12, 32)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 12, 12, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 12, 12, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 6, 6, 64)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 642,935 (2.45 MB)

 Trainable params: 641,463 (2.45 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [20]:
input_shape = (48, 48, 1)
num_classes = 7
batch_size = 32
num_epochs = 100
patience = 50
base_path = 'models/emotion_model'

faces, emotions = load_fer2013()
faces = preprocess_input(faces)

xtrain, xtest, ytrain, ytest = train_test_split(faces, emotions, test_size=0.2, shuffle=True)

data_generator = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    horizontal_flip=True
)

model = simple_CNN(input_shape, num_classes)
# لو عايز تجرب simple_CNN بدلها:
# model = simple_CNN(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    CSVLogger(base_path + '.log'),
    ModelCheckpoint(base_path + '.keras', monitor='val_loss', save_best_only=True, verbose=1)
,
    EarlyStopping(monitor='val_loss', patience=patience),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=int(patience / 4), verbose=1)
]

model.fit(
    data_generator.flow(xtrain, ytrain, batch_size=batch_size),
    steps_per_epoch=len(xtrain) // batch_size,
    epochs=num_epochs,
    verbose=1,
    callbacks=callbacks,
    validation_data=(xtest, ytest)
)

C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100


C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.2392 - loss: 1.8842
Epoch 1: val_loss improved from inf to 1.60569, saving model to models/emotion_model.keras
897/897 ━━━━━━━━━━━━━━━━━━━━ 112s 117ms/step - accuracy: 0.2392 - loss: 1.8841 - val_accuracy: 0.3759 - val_loss: 1.6057 - learning_rate: 0.0010
Epoch 2/100
  1/897 ━━━━━━━━━━━━━━━━━━━━ 1:53 126ms/step - accuracy: 0.1562 - loss: 1.7934

C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_loss did not improve from 1.60569
897/897 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.1562 - loss: 1.7934 - val_accuracy: 0.3695 - val_loss: 1.6114 - learning_rate: 0.0010
Epoch 3/100
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.3320 - loss: 1.6766
Epoch 3: val_loss improved from 1.60569 to 1.54451, saving model to models/emotion_model.keras
897/897 ━━━━━━━━━━━━━━━━━━━━ 99s 110ms/step - accuracy: 0.3320 - loss: 1.6766 - val_accuracy: 0.4036 - val_loss: 1.5445 - learning_rate: 0.0010
Epoch 4/100
  1/897 ━━━━━━━━━━━━━━━━━━━━ 1:33 104ms/step - accuracy: 0.3438 - loss: 1.5951
Epoch 4: val_loss did not improve from 1.54451
897/897 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.3438 - loss: 1.5951 - val_accuracy: 0.4009 - val_loss: 1.5467 - learning_rate: 0.0010
Epoch 5/100
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.3599 - loss: 1.6134
Epoch 5: val_loss improved from 1.54451 to 1.52419, saving model to models/emotion_model.keras
897/897 ━━━━━━━━━━━━

In [21]:
model.save(base_path + '_final.keras')

In [22]:
val_loss, val_accuracy = model.evaluate(xtest, ytest, verbose=0)
print(f"\n✅ Final Validation Accuracy: {val_accuracy * 100:.2f}%")


✅ Final Validation Accuracy: 58.18%


In [23]:
import cv2
import imutils
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np

# Load saved model
emotion_model_path = base_path + '_final.keras'
emotion_classifier = load_model(emotion_model_path)
EMOTIONS = ["angry", "disgust", "fear", "happy", "sad", "surprised", "neutral"]

# Load face detector
face_detector = cv2.CascadeClassifier('haarcascade_files/haarcascade_frontalface_default.xml')

cv2.namedWindow('Emotion Detector', cv2.WINDOW_NORMAL)
camera = cv2.VideoCapture(0)

while True:
    ret, frame = camera.read()
    if not ret:
        break

    frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    frameClone = frame.copy()

    if len(faces) > 0:
        (fX, fY, fW, fH) = sorted(faces, reverse=True, key=lambda x: x[2] * x[3])[0]
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        preds = emotion_classifier.predict(roi, verbose=0)[0]
        label = EMOTIONS[preds.argmax()]

        cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH), (147, 112, 219), 3)
        cv2.putText(frameClone, label.upper(), (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

    cv2.rectangle(frameClone, (0, 0), (500, 40), (50, 50, 50), -1)
    cv2.putText(frameClone, "Real-Time Emotion Recognition", (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 255, 255), 2)
    cv2.imshow('Emotion Detector', frameClone)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [24]:
model.save('trained_emotion_model.keras')  # Saved in the root folder